In [ ]:
#important libraries
!pip install httpx sgp4 poliastro 

#for exporting plots 
!pip install czml3

In [ ]:
#poliastro part

# From https://github.com/poliastro/poliastro/blob/main/contrib/satgpio.py
"""
Author: Juan Luis Cano Rodríguez

Code to read GP data from Celestrak using the HTTP API and python-sgp4.

Requires some extra dependencies:

  $ pip install httpx sgp4

This is similar to https://gitlab.com/librespacefoundation/python-satellitetle,
but uses the XML API instead and returns a `Satrec` object from sgp4 directly.

"""
#For loader
import io
import json
import xml.etree.ElementTree as ET

import httpx
from sgp4 import exporter, omm
from sgp4.api import Satrec




def _generate_url(catalog_number, international_designator, name):
    params = {
        "CATNR": catalog_number,
        "INTDES": international_designator,
        "NAME": name,
    }
    param_names = [
        param_name
        for param_name, param_value in params.items()
        if param_value is not None
    ]
    if len(param_names) != 1:
        raise ValueError(
            "Specify exactly one of catalog_number, international_designator, or name"
        )
    param_name = param_names[0]
    param_value = params[param_name]
    url = (
        "https://celestrak.org/NORAD/elements/gp.php?"
        f"{param_name}={param_value}"
        "&FORMAT=XML"
    )
    return url


def _segments_from_query(url):
    response = httpx.get(url)
    response.raise_for_status()

    if response.text == "No GP data found":
        raise ValueError(
            f"Query '{url}' did not return any results, try a different one"
        )
    tree = ET.parse(io.StringIO(response.text))
    root = tree.getroot()

    yield from omm.parse_xml(io.StringIO(response.text))


def load_gp_from_celestrak(
    *, catalog_number=None, international_designator=None, name=None
):
    """Load general perturbations orbital data from Celestrak.

    Returns
    -------
    Satrec
        Orbital data from specified object.

    Notes
    -----
    This uses the OMM XML format from Celestrak as described in [1]_.

    References
    ----------
    .. [1] Kelso, T.S. "A New Way to Obtain GP Data (aka TLEs)"
       https://celestrak.org/NORAD/documentation/gp-data-formats.php

    """
    # Assemble query, raise an error if malformed
    url = _generate_url(catalog_number, international_designator, name)

    # Make API call, raise an error if data is malformed
    for segment in _segments_from_query(url):
        # Initialize and return Satrec object
        sat = Satrec()
        omm.initialize(sat, segment)

        yield sat


def print_sat(sat, name):
    """Prints Satrec object in convenient form."""
    print(json.dumps(exporter.export_omm(sat, name), indent=2))

In [ ]:
from warnings import warn

from astropy.coordinates import TEME, GCRS

from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.czml.extract_czml import CZMLExtractor

def ephem_from_gp(sat, times):
    errors, rs, vs = sat.sgp4_array(times.jd1, times.jd2)
    if not (errors == 0).all():
        warn(
            "Some objects could not be propagated, "
            "proceeding with the rest",
            stacklevel=2,
        )
        rs = rs[errors == 0]
        vs = vs[errors == 0]
        times = times[errors == 0]

    cart_teme = CartesianRepresentation(
        rs << u.km,
        xyz_axis=-1,
        differentials=CartesianDifferential(
            vs << (u.km / u.s),
            xyz_axis=-1,
        ),
    )
    cart_gcrs = (
        TEME(cart_teme, obstime=times)
        .transform_to(GCRS(obstime=times))
        .cartesian
    )

    return Ephem(cart_gcrs, times, plane=Planes.EARTH_EQUATOR)

In [ ]:
from astropy import units as u
from astropy.time import Time

now = Time.now()
#now.jd1, now.jd2

import numpy as np

from astropy.coordinates import CartesianRepresentation, CartesianDifferential

from poliastro.util import time_range

In [ ]:
# Earth focused modules, ISS example orbit and time span generator
from poliastro.earth import EarthSatellite
from poliastro.earth.plotting import GroundtrackPlotter
from poliastro.util import time_range
from poliastro.twobody.orbit import Orbit
from poliastro.bodies import Earth

from poliastro.plotting import OrbitPlotter3D

import matplotlib.colors as mcolors

def loader(call, hours):
    palautettava={}
    palautettava["name"]=call
    palautettava["gp"] = list(load_gp_from_celestrak(name=call))[0]
    palautettava["ephem"] = ephem_from_gp(palautettava["gp"], time_range(start, end=start + (hours << u.h)))
    palautettava["orbit"]=Orbit.from_ephem(Earth,palautettava["ephem"],start)
    return palautettava

def spacecraft_instance_builder(sat,minutes):
    palautettava={}
    palautettava["orbit"]=EarthSatellite(sat["orbit"], None)
    palautettava["span"]=time_range(start, periods=150, end=start + minutes * u.min)
    return palautettava
    
    
def plottain(satelliitit:dict):
    plotter = OrbitPlotter3D()
    plotter.set_attractor(Earth)
    for x in satelliitit.values():
        plotter.plot_ephem(x["ephem"], label=x["name"], trail=True)
      
    return (plotter)

def GRplottain(satelliitit:dict, minutes=60):
    varit=[]
    i=0
    
    for x in satelliitit:
        
        varit.append(list(mcolors.CSS4_COLORS)[(i+1)*3])#Hypitään värejä kolmen välein että on vähän enempi kontrastia
        i+=1
    gp = GroundtrackPlotter()

    gp.update_layout(title="Groundplots")
    i=0
    for x in satelliitit.values():
        s1 =x["name"]
        s2 = varit[i]
        y=spacecraft_instance_builder(x,minutes) 
        gp.plot(y["orbit"],y["span"],label="%s %s"%(s1,s2),
        color=s2,#Tämä ei jostain systä toimi           
        marker={
        "size": 10,
        "symbol": "triangle-right",
        "line": {"width": 1, "color": "black"},},)
        #print(varit[i])
        
        i+=1
    return(gp)
        


In [ ]:
----------------------
#
#Syötä tähän SATNAME arvoja https://www.space-track.org/#catalog niistä satelliiteista mitä haluat plotata ja kuinka pitkä pätkä tunneissa

In [ ]:


start=now - 120 * u.day


In [ ]:


satellites = {}

#Molnyan


start=now - 120 * u.day

satellites["MOLNIYA 2-10"]=loader("MOLNIYA 2-10", 12)
satellites["MOLNIYA 1-88"]=loader("MOLNIYA 1-88", 12)
satellites["MOLNIYA 1-90"]=loader("MOLNIYA 1-90", 12)
satellites["MOLNIYA 1-87"]=loader("MOLNIYA 1-87", 12)
satellites["MOLNIYA 3-42"]=loader("MOLNIYA 3-42", 12)
satellites["MOLNIYA 3-42"]=loader("MOLNIYA 3-42", 12)
satellites["MOLNIYA 1-81"]=loader("MOLNIYA 1-81", 12)
satellites["MOLNIYA 2-14"]=loader("MOLNIYA 2-14", 12)
satellites["MOLNIYA 1-40"]=loader("MOLNIYA 1-40", 12)
satellites["MOLNIYA 1-52"]=loader("MOLNIYA 1-52", 12)
satellites["MOLNIYA 1-56"]=loader("MOLNIYA 1-56", 12)
satellites["MOLNIYA 1-63"]=loader("MOLNIYA 1-63", 12)
satellites["MOLNIYA 1-75"]=loader("MOLNIYA 1-75", 12)
satellites["MOLNIYA 1-80"]=loader("MOLNIYA 1-80", 12)
satellites["MOLNIYA 3-40"]=loader("MOLNIYA 3-40", 12)
satellites["MOLNIYA 1-9

start=now - 120 * u.day
1"]=loader("MOLNIYA 1-91", 12)
satellites["MOLNIYA 2-9"]=loader("MOLNIYA 2-9", 12)
satellites["MOLNIYA 1-S"]=loader("MOLNIYA 1-S", 12)
satellites["MOLNIYA 1-29"]=loader("MOLNIYA 1-29", 12)
satellites["MOLNIYA 1-32"]=loader("MOLNIYA 1-32", 12)
satellites["MOLNIYA 1-36"]=loader("MOLNIYA 1-36", 12)
satellites["MOLNIYA 3-8"]=loader("MOLNIYA 3-8", 12)
satellites["MOLNIYA 3-10"]=loader("MOLNIYA 3-10", 12)
satellites["MOLNIYA 3-13"]=loader("MOLNIYA 3-13", 12)
satellites["MOLNIYA 1-53"]=loader("MOLNIYA 1-53", 12)
satellites["MOLNIYA 3-20"]=loader("MOLNIYA 3-20", 12)
satellites["MOLNIYA 3-24"]=loader("MOLNIYA 3-24", 12)
satellites["MOLNIYA 3-27"]=loader("MOLNIYA 3-27", 12)
satellites["MOLNIYA 1-69"]=loader("MOLNIYA 1-69", 12)
satellites["MOLNIYA 3-31"]=loader("MOLNIYA 3-31", 12)
satellites["MOLNIYA 1-71"]=loader("MOLNIYA 1-71", 12)
satellites["MOLNIYA 3-41"]=loader("MOLNIYA 3-41", 12)
satellites["MOLNIYA 1-86"]=loader("MOLNIYA 1-86", 12)
satellites["MOLNIYA 3-47"]=loader("MOLNIYA 3-47", 12)
satellites["MOLNIYA 3-50"]=loader("MOLNIYA 3-50", 12)

In [ ]:
satellites = {}

#TDRS
satellites["TDRS 1"]=loader("TDRS 1", 12)
satellites["TDRS 3"]=loader("TDRS 3", 12)
satellites["TDRS 4"]=loader("TDRS 4", 12)
satellites["TDRS 5"]=loader("TDRS 5", 12)
satellites["TDRS 6"]=loader("TDRS 6", 12)
satellites["TDRS 7"]=loader("TDRS 7", 12)
satellites["TDRS 8"]=loader("TDRS 8", 12)
satellites["TDRS 9"]=loader("TDRS 9", 12)
satellites["TDRS 10"]=loader("TDRS 10", 12)
satellites["TDRS 11"]=loader("TDRS 11", 12)
satellites["TDRS 12"]=loader("TDRS 12", 12)
satellites["TDRS 13"]=loader("TDRS 13", 12)
satellites["TDRS 13"]['gp']

print_sat(satellites["TDRS 13"]['gp'],"TDRS 13")

In [ ]:
satellites = {}

#Unseen
satellites["BRO-1"]=loader("BRO-1", 2)
satellites["BRO-2"]=loader("BRO-2", 2)
satellites["BRO-3"]=loader("BRO-3", 2)
satellites["BRO-4"]=loader("BRO-4", 2)
satellites["BRO-5"]=loader("BRO-5", 2)
satellites["BRO-6"]=loader("BRO-6", 2)
satellites["BRO-7"]=loader("BRO-7", 2)
satellites["BRO-8"]=loader("BRO-8", 2)

print_sat(satellites["BRO-1"]['gp'],"BRO-1")
print(satellites["BRO-1"]['orbit'])

In [ ]:

satellites = {}


#satellites["Hawk-1"]=loader("Hawk-1", 2)
#satellites["Hawk-2"]=loader("Hawk-2", 2)
#satellites["Hawk-3"]=loader("Hawk-3", 2)
#satellites["Hawk-4"]=loader("Hawk-4", 2)
satellites["HAWK-A"]=loader("HAWK-A", 2)
satellites["HAWK-B"]=loader("HAWK-B", 2)
satellites["HAWK-C"]=loader("HAWK-C", 2)
satellites["HAWK-2A"]=loader("HAWK-2A", 2)
satellites["HAWK-2B"]=loader("HAWK-2B", 2)
satellites["HAWK-2C"]=loader("HAWK-2C", 2)
satellites["HAWK-3A"]=loader("HAWK-3A", 2)
satellites["HAWK-3B"]=loader("HAWK-3B", 2)
satellites["HAWK-3C"]=loader("HAWK-3C", 2)
satellites["HAWK-4A"]=loader("HAWK-4A", 2)
satellites["HAWK-4B"]=loader("HAWK-4B", 2)
satellites["HAWK-4C"]=loader("HAWK-4C", 2)
satellites["HAWK-5A"]=loader("HAWK-5A", 2)
satellites["HAWK-5B"]=loader("HAWK-5B", 2)
satellites["HAWK-5C"]=loader("HAWK-5C", 2)
#satellites["HAWK-6A"]=loader("HAWK-6A", 2)
#satellites["HAWK-6B"]=loader("HAWK-6B", 2)
#satellites["HAWK-6C"]=loader("HAWK-6C", 2)

In [ ]:
satellites = {}

#Capellat
satellites["CAPELLA-3-WHITNEY"]=loader("CAPELLA-3-WHITNEY", 2)
satellites["CAPELLA-4-WHITNEY"]=loader("CAPELLA-4-WHITNEY", 2)
satellites["CAPELLA-5-WHITNEY"]=loader("CAPELLA-5-WHITNEY", 2)
satellites["CAPELLA-6-WHITNEY"]=loader("CAPELLA-6-WHITNEY", 2)
satellites["CAPELLA-8-WHITNEY"]=loader("CAPELLA-3-WHITNEY", 2)

In [ ]:
satellites = {}

#ICEYEt

satellites["ICEYE-X1"]=loader("ICEYE-X1", 1)
satellites["ICEYE-X2"]=loader("ICEYE-X2", 1)
satellites["ICEYE-X4"]=loader("ICEYE-X4", 1)
satellites["ICEYE-X5"]=loader("ICEYE-X5", 1)
satellites["ICEYE-X6"]=loader("ICEYE-X6", 1)
satellites["ICEYE-X7"]=loader("ICEYE-X7", 1)
satellites["ICEYE-X8"]=loader("ICEYE-X8", 1)
satellites["ICEYE-X9"]=loader("ICEYE-X9", 1)
satellites["ICEYE-X11"]=loader("ICEYE-X11", 1)
satellites["ICEYE-X12"]=loader("ICEYE-X12", 1)
satellites["ICEYE-X13"]=loader("ICEYE-X13", 1)
satellites["ICEYE-X14"]=loader("ICEYE-X14", 1)
satellites["ICEYE-X15"]=loader("ICEYE-X15", 1)
satellites["ICEYE-X16"]=loader("ICEYE-X16", 1)
satellites["ICEYE-X17"]=loader("ICEYE-X17", 1)
satellites["ICEYE-X18"]=loader("ICEYE-X18", 1)
satellites["ICEYE-X19"]=loader("ICEYE-X19", 1)
satellites["ICEYE-X20"]=loader("ICEYE-X20", 1)
satellites["ICEYE-X24"]=loader("ICEYE-X24", 1)

print_sat(satellites["ICEYE-X11"]['gp'],"ICEYE-X11")

In [ ]:
satellites = {}

#Sentinelit

satellites["SENTINEL 1A"]=loader("SENTINEL-1A", 1)

satellites["SENTINEL 1B"]=loader("SENTINEL-1B", 1)
satellites["SENTINEL 2A"]=loader("SENTINEL-2A", 1)
satellites["SENTINEL 2B"]=loader("SENTINEL-2B", 1)
satellites["SENTINEL 3A"]=loader("SENTINEL-3A", 1)
satellites["SENTINEL 3B"]=loader("SENTINEL-3B", 1)
satellites["SENTINEL 5P"]=loader("SENTINEL-5P", 1)

satellites["SENTINEL 1A"] #Near frozen orbit 

In [ ]:
satellites = {}

#Galileo
satellites["GSAT0101"]=loader("GSAT0101", 12)
satellites["GSAT0102"]=loader("GSAT0102", 12)
satellites["GSAT0103"]=loader("GSAT0103", 12)
satellites["GSAT0104"]=loader("GSAT0104", 12)
satellites["GSAT0201"]=loader("GSAT0201", 12)
satellites["GSAT0104"]=loader("GSAT0104", 12)
satellites["GSAT0201"]=loader("GSAT0201", 12)
satellites["GSAT0202"]=loader("GSAT0202", 12)
satellites["GSAT0203"]=loader("GSAT0203", 12)
satellites["GSAT0204"]=loader("GSAT0204", 12)
satellites["GSAT0205"]=loader("GSAT0205", 12)
satellites["GSAT0206"]=loader("GSAT0206", 12)
satellites["GSAT0207"]=loader("GSAT0207", 12)
satellites["GSAT0208"]=loader("GSAT0208", 12)
satellites["GSAT0209"]=loader("GSAT0209", 12)
satellites["GSAT0210"]=loader("GSAT0210", 12)
satellites["GSAT0211"]=loader("GSAT0211", 12)
satellites["GSAT0212"]=loader("GSAT0212", 12)
satellites["GSAT0213"]=loader("GSAT0213", 12)
satellites["GSAT0214"]=loader("GSAT0214", 12)
satellites["GSAT0215"]=loader("GSAT0215", 12)
satellites["GSAT0216"]=loader("GSAT0216", 12)
satellites["GSAT0217"]=loader("GSAT0217", 12)
satellites["GSAT0218"]=loader("GSAT0218", 12)
satellites["GSAT0219"]=loader("GSAT0219", 12)
satellites["GSAT0220"]=loader("GSAT0220", 12)
satellites["GSAT0221"]=loader("GSAT0221", 12)
satellites["GSAT0222"]=loader("GSAT0222", 12)
satellites["GSAT0223"]=loader("GSAT0223", 12)
satellites["GSAT0224"]=loader("GSAT0224", 12)

print_sat(satellites["GSAT0101"]['gp'],"GSAT0101")

In [ ]:
#For printing the orbital parameters of the satellites
for key in satellites:
  print_sat(satellites[str(key)]['gp'],str(key))
  print(satellites[str(key)]['orbit'])

In [ ]:


kolmiplotti= plottain(satellites)

kolmiplotti.show()


In [ ]:

maatrakki_minuutit=1*60
maatrakki=GRplottain(satellites,maatrakki_minuutit,)
maatrakki.fig.show()

In [ ]:
maatrakki.update_geos(projection_type="orthographic")
maatrakki.fig.show()
